In [30]:
!pip install contractions
!pip install emot
!pip install num2words
!pip install afinn
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
#Obtaining the datasets
movies_train = pd.read_csv('train.tsv', sep='\t')
movies_test = pd.read_csv('test.tsv', sep='\t')

In [8]:
import nltk
from nltk.corpus import gutenberg, stopwords, wordnet
import contractions
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import re
from bs4 import BeautifulSoup
from num2words import num2words
from emot.emo_unicode import UNICODE_EMOJI #for EMOJIS
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
def lower_case(text):
  return text.lower()
def rem_lines(text):
    return text.strip().replace('\n', ' ')
def rem_tags(text):
  return BeautifulSoup(text, "html.parser").get_text()
def fix_ct(text):
    return contractions.fix(text)
def rem_stopwords(text):
  stop_words = set(stopwords.words('english'))
  return [word for word in word_tokenize(text) if not word in stop_words]
def rem_punct(text):
    no_punct = [w.translate(str.maketrans('', '', string.punctuation)) for w in word_tokenize(text)]
    return [word for word in no_punct if word.isalpha()]
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
def to_number(text):
    return(re.sub(r"(\d+)", lambda x: num2words(int(x.group(0))), text))
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, EMOTICONS_EMO[emot]+" ".replace("",""))
    return text
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot,UNICODE_EMOJI[emot]+" ".replace("",""))
    return text
def clean_text(text):
  text = text.lower() # convert to lowercase
  text=re.sub(r'[@#]', '', text) #remove tags
  return text

In [10]:
movies_train['sentence']=movies_train['Phrase'].astype(str)
#cleaning sentence
movies_train['sentence']=movies_train['sentence'].astype(str).apply(lower_case)
movies_train['sentence']=movies_train['sentence'].astype(str).apply(rem_lines)
movies_train['sentence']=movies_train['sentence'].astype(str).apply(rem_tags)
#movies_train['sentence']=movies_train['sentence'].astype(str).apply(convert_emoticons)
#movies_train['sentence']=movies_train['sentence'].astype(str).apply(convert_emojis)
#movies_train['sentence']=movies_train['sentence'].astype(str).apply(fix_ct)
#movies_train['sentence']=movies_train['sentence'].astype(str).apply(rem_stopwords)
movies_train['sentence']=movies_train['sentence'].astype(str).apply(rem_punct)
#movies_train['sentence']=movies_train['sentence'].astype(str).apply(to_number)
movies_train['sentence']=movies_train['sentence'].astype(str).apply(lemmatize_words)

<ipython-input-9-1ee3cf2cb3f9>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


In [11]:
movies_test['sentence']=movies_test['Phrase'].astype(str)
#cleaning sentence
movies_test['sentence']=movies_test['sentence'].astype(str).apply(lower_case)
movies_test['sentence']=movies_test['sentence'].astype(str).apply(rem_lines)
movies_test['sentence']=movies_test['sentence'].astype(str).apply(rem_tags)
#movies_test['sentence']=movies_test['sentence'].astype(str).apply(convert_emoticons)
#movies_test['sentence']=movies_test['sentence'].astype(str).apply(convert_emojis)
#movies_test['sentence']=movies_test['sentence'].astype(str).apply(fix_ct)
#movies_test['sentence']=movies_test['sentence'].astype(str).apply(rem_stopwords)
movies_test['sentence']=movies_test['sentence'].astype(str).apply(rem_punct)
#movies_test['sentence']=movies_test['sentence'].astype(str).apply(to_number)
movies_test['sentence']=movies_test['sentence'].astype(str).apply(lemmatize_words)

<ipython-input-9-1ee3cf2cb3f9>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


In [12]:
#sentiment analysis using afinn
from afinn import Afinn
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as CR
afn = Afinn()
scores = [afn.score(article) for article in movies_train['sentence']]
print(scores)

[6.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 2.0, 2.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 0.0, -2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, 0.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.0, -8.0, -6.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, -8.0, -8.0, 0.0, 0.0, -8.0, 0.0, -8.0, -8.0, 0.0, -8.0, -8.0, -3.0, -5.0, -5.0, -3.0, -3.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 0.0, -2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.0, -5.0, -2.0, -3.0, 0.0, 0.0, -3.0, -3.0, 0.0, -3.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, 2.0, 0.0, 2.0, 0.0, 2.0, 2.0, 2.0, 0.0, -3.0, 0.0, -3.0, 0.0, 0.

In [13]:
#getting sentiments
def get_sentiment(scores):
  sentiment = [0 if score < -4
              else 1 if score < -0.5
              else 2 if score < 0.5
              else 3 if score < 4
              else 4
              for score in scores]
  return sentiment
sentiment=get_sentiment(scores)
print(CR(movies_train['Sentiment'],sentiment))
accuracy = accuracy_score(movies_train['Sentiment'],sentiment)
print("Accuracy:", accuracy)#Accuracy too low

              precision    recall  f1-score   support

           0       0.30      0.07      0.11      7072
           1       0.36      0.31      0.33     27273
           2       0.65      0.75      0.69     79582
           3       0.39      0.37      0.38     32927
           4       0.28      0.26      0.27      9206

    accuracy                           0.53    156060
   macro avg       0.40      0.35      0.36    156060
weighted avg       0.51      0.53      0.51    156060

Accuracy: 0.5305779828271178


In [14]:
scores = [afn.score(article) for article in movies_test['sentence']]
sentiments_afn=get_sentiment(scores)
df_afn=pd.DataFrame(movies_test['PhraseId'])
df_afn['Sentiment']=sentiments_afn
print(df_afn)

       PhraseId  Sentiment
0        156061          2
1        156062          2
2        156063          2
3        156064          2
4        156065          2
...         ...        ...
66287    222348          2
66288    222349          2
66289    222350          2
66290    222351          2
66291    222352          2

[66292 rows x 2 columns]


In [15]:
# Training data preparation
from sklearn.model_selection import train_test_split
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(movies_train['sentence'], movies_train['Sentiment'], test_size=0.2, random_state=42)
#quick visualization
print(X_train)
print(X_test)

22538                                            ['cheesy']
99237                            ['unintentionally', 'rrb']
60377     ['will', 'need', 'all', 'the', 'luck', 'they',...
128317    ['somewhere', 'between', 'sling', 'blade', 'an...
20776     ['reminds', 'at', 'every', 'turn', 'of', 'eliz...
                                ...                        
119879                      ['its', 'trademark', 'villain']
103694    ['earn', 'her', 'share', 'of', 'the', 'holiday...
131932    ['a', 'moving', 'tale', 'of', 'love', 'and', '...
146867    ['if', 'you', 'love', 'reading', 'andor', 'poe...
121958            ['and', 'drag', 'audience', 'enthusiasm']
Name: sentence, Length: 124848, dtype: object
95722                                  ['s', 'as', 'sorry']
147312    ['romantic', 'comedy', 'and', 'dogme', 'filmma...
36991                               ['of', 'these', 'days']
150211    ['flinch', 'from', 'its', 'unsettling', 'progn...
140655                   ['are', 'clinically', 'depres

In [16]:
#Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
#Vectorizer
vectorizer = TfidfVectorizer(min_df=0., max_df=1., norm='l2',
 use_idf=True, smooth_idf=True)
#Training data vectorization
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [17]:
#Problem vectorization
final_test_vec=vectorizer.transform(movies_test['sentence'].astype(str))
#total training vectorization
final_train=vectorizer.transform(movies_train['sentence'].astype(str))

#Training

In [18]:
#Some models
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.svm import LinearSVC as SVM
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import classification_report as CR
from sklearn.naive_bayes import BernoulliNB as NB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import Perceptron as PT
#initial models configuration
svm=SVM(penalty='l2',max_iter=500,C=1,random_state=42)
svc=SVC(max_iter=500,C=1,random_state=42)
sgd=SGD(loss='hinge',penalty='l2',max_iter=500,random_state=42)
lr=LR(max_iter=500)
nb=NB(alpha=.1)
rfc=RFC(n_estimators=10,random_state=42)
pt=PT(tol=1e-3, random_state=42)
#array of models
models=[svm,sgd,lr, nb, rfc, pt]
#array of results

In [19]:
#Training with each model
from sklearn.model_selection import cross_val_score
def fit(x_train,y_train,x_test,y_test):
  for model in models:
    model.fit(x_train,y_train)
    print(model)
    prediction=(model.predict(x_test))
    print(CR(y_test,prediction))
    scores = cross_val_score(model, x_train, y_train, cv=5)
    mean_accuracy = scores.mean()
    print("mean cross_validation score",mean_accuracy)
#function execution
fit(X_train_vec,y_train,X_test_vec,y_test)

LinearSVC(C=1, max_iter=500, random_state=42)
              precision    recall  f1-score   support

           0       0.53      0.29      0.38      1416
           1       0.54      0.45      0.49      5527
           2       0.70      0.84      0.76     15639
           3       0.57      0.51      0.54      6707
           4       0.57      0.34      0.43      1923

    accuracy                           0.64     31212
   macro avg       0.58      0.49      0.52     31212
weighted avg       0.63      0.64      0.63     31212

mean cross_validation score 0.6371587900755046
SGDClassifier(max_iter=500, random_state=42)
              precision    recall  f1-score   support

           0       0.57      0.09      0.16      1416
           1       0.48      0.12      0.19      5527
           2       0.56      0.95      0.71     15639
           3       0.52      0.21      0.29      6707
           4       0.57      0.12      0.20      1923

    accuracy                           0.55    

In [32]:
#Hyperparameter tunned
from sklearn.metrics import accuracy_score
svm2= SVM(penalty='l2',max_iter=1000,C=10,random_state=42, tol=1e-9)
models=[svm]
svm2.fit(X_train_vec,y_train)
prediction=(svm.predict(X_test_vec))
print(CR(y_test,prediction))
print(accuracy_score(y_test,prediction))

              precision    recall  f1-score   support

           0       0.47      0.34      0.40      1416
           1       0.53      0.48      0.50      5527
           2       0.71      0.81      0.76     15639
           3       0.56      0.51      0.54      6707
           4       0.53      0.40      0.45      1923

    accuracy                           0.64     31212
   macro avg       0.56      0.51      0.53     31212
weighted avg       0.63      0.64      0.63     31212

0.6394976291170062


#

In [21]:
#training with the totality of the traning data
svm.fit(final_train,movies_train['Sentiment'])
y_pred=(svm.predict(final_test_vec))
submit=pd.DataFrame(movies_test['PhraseId'])
submit['Sentiment']=pd.Series(y_pred)
print(submit)

       PhraseId  Sentiment
0        156061          3
1        156062          3
2        156063          2
3        156064          3
4        156065          3
...         ...        ...
66287    222348          1
66288    222349          1
66289    222350          1
66290    222351          1
66291    222352          2

[66292 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [33]:
#training with better version
svm2.fit(final_train,movies_train['Sentiment'])
y_pred=(svm2.predict(final_test_vec))
submit2=pd.DataFrame(movies_test['PhraseId'])
submit2['Sentiment']=pd.Series(y_pred)
print(submit2)

       PhraseId  Sentiment
0        156061          3
1        156062          3
2        156063          2
3        156064          3
4        156065          3
...         ...        ...
66287    222348          1
66288    222349          1
66289    222350          1
66290    222351          1
66291    222352          2

[66292 rows x 2 columns]


#Submission

In [23]:
from google.colab import files
#csv creation and download
file_name_csv = "submission_svm.csv"  # Nombre del archivo CSV de salida
submit.to_csv(file_name_csv, index=False)  # index=False para omitir la columna de índices
files.download(file_name_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
file_name_csv = "submission_afin.csv"
df_afn.to_csv(file_name_csv,index=False)
files.download(file_name_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
file_name_csv = "submission_second_svm.csv"
submit2.to_csv(file_name_csv,index=False)
files.download(file_name_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>